<a href="https://colab.research.google.com/github/22cs044Shruti/C-/blob/main/Practical_3_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [3]:
churn_data = pd.read_csv("/content/customer_churn_dataset-testing-master.csv")

In [4]:
churn_data

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,1,22,Female,25,14,4,27,Basic,Monthly,598,9,1
1,2,41,Female,28,28,7,13,Standard,Monthly,584,20,0
2,3,47,Male,27,10,2,29,Premium,Annual,757,21,0
3,4,35,Male,9,12,5,17,Premium,Quarterly,232,18,0
4,5,53,Female,58,24,9,2,Standard,Annual,533,18,0
...,...,...,...,...,...,...,...,...,...,...,...,...
64369,64370,45,Female,33,12,6,21,Basic,Quarterly,947,14,1
64370,64371,37,Male,6,1,5,22,Standard,Annual,923,9,1
64371,64372,25,Male,39,14,8,30,Premium,Monthly,327,20,1
64372,64373,50,Female,18,19,7,22,Standard,Monthly,540,13,1


Preprocessing

In [5]:
churn_data['Gender'] = LabelEncoder().fit_transform(churn_data['Gender'])
churn_data = pd.get_dummies(churn_data, drop_first=True)

In [6]:
target = 'Churn'
X = churn_data.drop(target, axis=1)
y = churn_data[target]

In [7]:
X

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Total Spend,Last Interaction,Subscription Type_Premium,Subscription Type_Standard,Contract Length_Monthly,Contract Length_Quarterly
0,1,22,0,25,14,4,27,598,9,False,False,True,False
1,2,41,0,28,28,7,13,584,20,False,True,True,False
2,3,47,1,27,10,2,29,757,21,True,False,False,False
3,4,35,1,9,12,5,17,232,18,True,False,False,True
4,5,53,0,58,24,9,2,533,18,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64369,64370,45,0,33,12,6,21,947,14,False,False,False,True
64370,64371,37,1,6,1,5,22,923,9,False,True,False,False
64371,64372,25,1,39,14,8,30,327,20,True,False,True,False
64372,64373,50,0,18,19,7,22,540,13,False,True,True,False


In [8]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# random_state can be 28,64 also

In [10]:
y_train

,Churn
13468,0
20118,0
48411,1
42125,1
55119,1
...,...
62570,1
38158,0
860,1
15795,0


In [11]:
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

In [12]:
input_size =  X_train.shape[1] #for getting number of features
hidden_size = 10
output_size = 1

In [13]:
print(input_size)

13


In [14]:
# Initialize weights and bias
np.random.seed(42) # to get same weights
W1 = np.random.randn(input_size, hidden_size) * 0.01
b1 = np.zeros((1, hidden_size))
W2 = np.random.randn(hidden_size, output_size) * 0.01
b2 = np.zeros((1, output_size))

In [15]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
  return sigmoid(x) * (1 - sigmoid(x))


In [20]:
def ReLu(x):
  return np.maximum(0, x)

def ReLu_derivative(x):
  return np.where(x > 0, 1, 0)

In [21]:
def forward_propagation(X):
  Z1 = np.dot(X, W1) + b1
  A1 = ReLu(Z1)
  Z2 = np.dot(A1, W2) + b2
  A2 = sigmoid(Z2)
  return Z1, A1, Z2, A2

In [22]:
def backward_propagation(X, y, Z1, A1, Z2, A2):
  m = X.shape[0]

  dZ2 = A2 - y
  dW2 = np.dot(A1.T, dZ2) / m
  db2 = np.sum(dZ2, axis = 0, keepdims=True) / m

  dA1 = np.dot(dZ2, W2.T)
  dZ1 = dA1 * ReLu_derivative(Z1)
  dW1 = np.dot(X.T, dZ1) / m
  db1 = np.sum(dZ1, axis = 0, keepdims=True) / m

  return dW1, db1, dW2, db2

In [23]:
def train(X,y,learning_rate=0.01, epochs=100):
  global W1, b1, W2, b2

  for epoch in range(epochs):

    Z1, A1, Z2, A2 = forward_propagation(X)
    # Here A2 is output

    loss = -np.mean(y * np.log(A2) + (1 - y) * np.log(1 - A2))

    dW1, db1, dW2, db2 = backward_propagation(X, y, Z1, A1, Z2, A2)

    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2

    if epoch % 10 == 0:
      print(f"Epoch {epoch}, Loss: {loss}")

In [24]:
train(X_train, y_train, learning_rate=0.001, epochs=1000)

Epoch 0, Loss: 0.6931465773327827
Epoch 1, Loss: 0.6931392046955586
Epoch 2, Loss: 0.6931318651501375
Epoch 3, Loss: 0.6931245586118441
Epoch 4, Loss: 0.6931172848586813
Epoch 5, Loss: 0.6931100438532287
Epoch 6, Loss: 0.6931028353431602
Epoch 7, Loss: 0.6930956590914847
Epoch 8, Loss: 0.6930885148597289
Epoch 9, Loss: 0.6930814025129002
Epoch 10, Loss: 0.6930743219977391
Epoch 11, Loss: 0.6930672730266428
Epoch 12, Loss: 0.6930602554522243
Epoch 13, Loss: 0.6930532692670529
Epoch 14, Loss: 0.6930463144228108
Epoch 15, Loss: 0.6930393905191321
Epoch 16, Loss: 0.6930324974300581
Epoch 17, Loss: 0.693025634995973
Epoch 18, Loss: 0.6930188030279308
Epoch 19, Loss: 0.6930120013404424
Epoch 20, Loss: 0.6930052294353674
Epoch 21, Loss: 0.6929984873583537
Epoch 22, Loss: 0.6929917750056429
Epoch 23, Loss: 0.6929850923203394
Epoch 24, Loss: 0.6929784389726348
Epoch 25, Loss: 0.6929718149402319
Epoch 26, Loss: 0.6929652199633682
Epoch 27, Loss: 0.6929586538481685
Epoch 28, Loss: 0.6929521165129

In [ ]:
def predict(X):
  Z1, A1, Z2, A2 = forward_propagation(X)
  predictions = (A2 >= 0.5).astype(int)
  return predictions